In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df = pd.read_csv('ev_charging_patterns.csv')
df.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


Explore the data - do some basic KPIs/Stats

In [25]:
print("Dataset Shape:", df.shape )

Dataset Shape: (1320, 20)


In [26]:
print("\nColumn types:", df.dtypes)


Column types: User ID                                      object
Vehicle Model                                object
Battery Capacity (kWh)                      float64
Charging Station ID                          object
Charging Station Location                    object
Charging Start Time                          object
Charging End Time                            object
Energy Consumed (kWh)                       float64
Charging Duration (hours)                   float64
Charging Rate (kW)                          float64
Charging Cost (USD)                         float64
Time of Day                                  object
Day of Week                                  object
State of Charge (Start %)                   float64
State of Charge (End %)                     float64
Distance Driven (since last charge) (km)    float64
Temperature (°C)                            float64
Vehicle Age (years)                         float64
Charger Type                                 obje

In [27]:
print("\nMissing values:", df.isnull().sum())


Missing values: User ID                                      0
Vehicle Model                                0
Battery Capacity (kWh)                       0
Charging Station ID                          0
Charging Station Location                    0
Charging Start Time                          0
Charging End Time                            0
Energy Consumed (kWh)                       66
Charging Duration (hours)                    0
Charging Rate (kW)                          66
Charging Cost (USD)                          0
Time of Day                                  0
Day of Week                                  0
State of Charge (Start %)                    0
State of Charge (End %)                      0
Distance Driven (since last charge) (km)    66
Temperature (°C)                             0
Vehicle Age (years)                          0
Charger Type                                 0
User Type                                    0
dtype: int64


If there is no cost and distance data, the easiest will be to remove those rows as there is still another 1254 available datapoints

In [28]:
print("\nBasic Stats:", df.describe())


Basic Stats:        Battery Capacity (kWh)  Energy Consumed (kWh)  \
count             1320.000000            1254.000000   
mean                74.534692              42.642894   
std                 20.626914              22.411705   
min                  1.532807               0.045772   
25%                 62.000000              23.881193   
50%                 75.000000              42.691405   
75%                 85.000000              61.206218   
max                193.003074             152.238758   

       Charging Duration (hours)  Charging Rate (kW)  Charging Cost (USD)  \
count                1320.000000         1254.000000          1320.000000   
mean                    2.269377           25.963003            22.551352   
std                     1.061037           14.011326            10.751494   
min                     0.095314            1.472549             0.234317   
25%                     1.397623           13.856583            13.368141   
50%                

Battery Capacity (kWh); small standard deviation indicating the values are clustered together around the mean. A minimum of 1.53 kWh seems impossible for an EV battery, might have to remove that datapoint.

Energy Consumed (kWh) I do not see anything out of the ordinary with those figures.

Charging duration (hours); those figures seem standard. I'm not too sure of the maximum Charging duration being 7 hours; but investigating further I see the final SOC% was over 100% and the session start and end time only differs by two hours so these must be faulty readings

Charging Rate (kW); nothing seems out of the ordinary here. Same for Charging Cost (USD) but what I might have to do is instead remove this column and replace it with what it will cost to charge in south africa based on Charger type and time of day but that would be a lot of extra legwork/conditionals

State of Charge (Start & End); there are a few entries where the SOC% is greater than 100 which is impossible - so those can be removed. The Start SOC% makes ense to be spread out (large standard deviation) and the same for the end SOC% (small standard deviation becuase typically when charging the EVs the general consensus is to try and charge till full)

Distance Driven (since last charge) (km) makes sense to have a large standard deviation since it it is user dependent. For teh Temperautre I am unsure becuase I assume it's motor temperature.

Vehicle Age (years) it makes sense for the values not to be too spread out.



In [29]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\n Duplicate rows: {duplicates}")


 Duplicate rows: 0


Here I am going to try and fix the data quality issues

In [30]:
before_fix = df['Charger Type'].value_counts()
df['Charger Type'] = df['Charger Type'].replace('DC Fast', 'DC Fast Charger')
after_fix = df['Charger Type'].value_counts()

print(f"Before: {before_fix.to_dict()}")
print(f"After: {after_fix.to_dict()}")

Before: {'Level 1': 459, 'Level 2': 431, 'DC Fast Charger': 430}
After: {'Level 1': 459, 'Level 2': 431, 'DC Fast Charger': 430}


Hnadle the missing values - going to try and keep all of it but will flag any missing data.

In [31]:
df['has_distance_data'] = ~df['Distance Driven (since last charge) (km)'].isnull()
df['has_energy_data'] = ~df['Energy Consumed (kWh)'].isnull()

print(f"Rows with distance data: {df['has_distance_data'].sum()} ({df['has_distance_data'].sum()/len(df)*100:.2f}%)")
print(f"Rows with energy data: {df['has_energy_data'].sum()} ({df['has_energy_data'].sum()/len(df)*100:.2f}%)")

#Drop the entry missing a user Type
rows_before = len(df)
df = df.dropna(subset=['User Type'])
rows_after = len(df)
print(f"Rows before: {rows_before} | Rows after: {rows_after}")

Rows with distance data: 1254 (95.00%)
Rows with energy data: 1254 (95.00%)
Rows before: 1320 | Rows after: 1320


Create Cleaner Column names

In [40]:
column_mapping = {
    'User ID': 'user_id',
    'Vehicle Model': 'vehicle_model',
    'Battery Capacity (kWh)': 'battery_capacity_kwh',
    'Charging Station ID': 'station_id',
    'Charging Station Location': 'city',
    'Charging Start Time': 'start_time',
    'Charging End Time': 'end_time',
    'Energy Consumed (kWh)': 'energy_consumed_kwh',
    'Charging Duration (hours)': 'duration_hours',
    'Charging Rate (kW)': 'charging_rate_kw',
    'Charging Cost (USD)': 'cost_usd',
    'Time of Day': 'portion_of_day',
    'Day of Week': 'day_of_week',
    'State of Charge (Start %)': 'soc_start_pct',
    'State of Charge (End %)': 'soc_end_pct',
    'Distance Driven (since last charge) (km)': 'distance_km',
    'Temperature (°C)': 'temperature_c',
    'Vehicle Age (years)': 'vehicle_age',
    'Charger Type': 'charger_type',
    'User Type': 'user_type'
}
df = df.rename(columns = column_mapping)
df.head(5)

,user_id,vehicle_model,battery_capacity_kwh,station_id,city,start_time,end_time,energy_consumed_kwh,duration_hours,charging_rate_kw,...,day_of_week,soc_start_pct,soc_end_pct,distance_km,temperature_c,vehicle_age,charger_type,user_type,has_distance_data,has_energy_data
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,...,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter,True,True
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,...,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver,True,True
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,...,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter,True,True
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,...,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler,True,True
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,...,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler,True,True


Convert the data types

In [41]:
# Convert datetime columns
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
print(df.dtypes)


user_id                         object
vehicle_model                   object
battery_capacity_kwh           float64
station_id                      object
city                            object
start_time              datetime64[ns]
end_time                datetime64[ns]
energy_consumed_kwh            float64
duration_hours                 float64
charging_rate_kw               float64
cost_usd                       float64
time_of_day                     object
day_of_week                     object
soc_start_pct                  float64
soc_end_pct                    float64
distance_km                    float64
temperature_c                  float64
vehicle_age                    float64
charger_type                    object
user_type                       object
has_distance_data                 bool
has_energy_data                   bool
dtype: object


Now save the clean dataset

In [42]:
# Save to CSV
df.to_csv('ev_charging_patterns_CLEAN.csv', index=False)
print(" Saved to: ev_charging_patterns_CLEAN.csv")

# Also save a version ready for analysis (only rows with complete core data)
df_complete = df[df['has_distance_data'] & df['has_energy_data']].copy()
df_complete.to_csv('ev_charging_patterns_COMPLETE.csv', index=False)

 Saved to: ev_charging_patterns_CLEAN.csv
